# Metadata

**L1 Taxonomy** - Background Processes

**L2 Taxonomy** - Asynchronous Processing

**Subtopic** - Handling exceptions in asyncio tasks and collecting error results

**Use Case** - Implement a Python script that uses asyncio to perform asynchronous file operations. The script should be able to handle and collect exceptions from the asyncio tasks. This script should read multiple text files concurrently, count the number of lines in each file, and print the results. If a file cannot be read due to an exception (like file not found), the script should catch the exception, store the error message, and continue with the other files.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

You are building a distributed log analysis system that concurrently processes multiple server log files to count lines and generate statistics. The system must handle various file access issues, like missing files and permission errors while collecting results from successful operations and tracking failures for monitoring purposes.

## Input Format

The function receives a list of dictionaries, where each dictionary represents a log file configuration:

```
log_files = [
    {"file_id": "web-server-01", "file_path": "/logs/web.log", "timeout": 0.3, "expected_lines": 45, "error_type": None},
    {"file_id": "api-gateway", "file_path": "/logs/api.log", "timeout": 0.1, "expected_lines": 32, "error_type": "file_not_found"},
    {"file_id": "database-01", "file_path": "/logs/db.log", "timeout": 0.5, "expected_lines": 67, "error_type": None},
    {"file_id": "cache-redis", "file_path": "/logs/cache.log", "timeout": 0.2, "expected_lines": 28, "error_type": "permission_denied"},
    {"file_id": "auth-service", "file_path": "/logs/auth.log", "timeout": 0.4, "expected_lines": 44, "error_type": None}
]
```

## Output Format

Returns a dictionary containing line count results and error tracking:

```
{
    "total_lines": 156,
    "successful_files": 3,
    "error_summary": {
        "FileNotFoundError": 1,
        "PermissionError": 1,
        "TimeoutError": 0
    },
    "file_results": [
        {"file_id": "web-server-01", "status": "success", "line_count": 45},
        {"file_id": "api-gateway", "status": "error", "error_type": "FileNotFoundError"},
        {"file_id": "database-01", "status": "success", "line_count": 67},
        {"file_id": "cache-redis", "status": "error", "error_type": "PermissionError"},
        {"file_id": "auth-service", "status": "success", "line_count": 44}
    ]
}
```

## Examples

Example 1:

Input:

```
log_files = [
    {"file_id": "srv1", "file_path": "/var/log/app.log", "timeout": 0.2, "expected_lines": 34, "error_type": None},
    {"file_id": "srv2", "file_path": "/var/log/err.log", "timeout": 0.3, "expected_lines": 28, "error_type": "file_not_found"},
    {"file_id": "srv3", "file_path": "/var/log/sys.log", "timeout": 0.1, "expected_lines": 55, "error_type": None}
]
```

Output:

```
{
    "total_lines": 89,
    "successful_files": 2,
    "error_summary": {"FileNotFoundError": 1, "PermissionError": 0, "TimeoutError": 0},
    "file_results": [
        {"file_id": "srv1", "status": "success", "line_count": 34},
        {"file_id": "srv2", "status": "error", "error_type": "FileNotFoundError"},
        {"file_id": "srv3", "status": "success", "line_count": 55}
    ]
}
```

Example 2:

Input:

```
log_files = [
    {"file_id": "monitor", "file_path": "/logs/mon.log", "timeout": 0.05, "expected_lines": 42, "error_type": timeout},
    {"file_id": "backup", "file_path": "/logs/bak.log", "timeout": 0.4, "expected_lines": 18, "error_type": "permission_denied"}
]
```

Output:

```
{
    "total_lines": 0,
    "successful_files": 0,
    "error_summary": {"FileNotFoundError": 0, "PermissionError": 1, "TimeoutError": 1},
    "file_results": [
        {"file_id": "monitor", "status": "error", "error_type": "TimeoutError"},
        {"file_id": "backup", "status": "error", "error_type": "PermissionError"}
    ]
}
```

# Requirements

## Explicit Requirements

- Process all log files concurrently using asynchronous operations with exception collection
- For successful files (error_type=None), return the expected_lines value as line_count
- Generate appropriate exceptions for different error scenarios during file operations
- Calculate total_lines by summing line_count from all successfully processed files
- Store exception type names in error tracking without preserving full exception objects
- Maintain original input order in the results array

## Implicit Requirements

- Operations must complete within reasonable time bounds considering individual timeouts
- The expected_lines parameter is only used for successful operations and ignored for failed ones
- All three error types must appear in the error summary regardless of occurrence
- File operations should simulate realistic reading delays and potential failures
- Empty input should produce meaningful zero-state results

## Function Signature

```python
def analyze_log_files(log_files: List[Dict[str, Any]]) -> Dict[str, Any]:
    pass
```

## Edge Cases

- Empty log_files list should return zero totals with empty file_results array
- All files succeed should show no errors and sum all expected_lines values
- All files fail should show zero total_lines and appropriate error counts
- Timeout value of 0.0 should immediately cause timeout for that file
- Duplicate file_id values should be processed independently
- Invalid error_type values should be treated as None and process successfully
- Negative timeout values should be treated as immediate timeout
- The expected_lines value should be ignored when file operation fails due to errors

## Constraints

- Maximum 50 log files per function call
- Timeout values between 0.0 and 2.0 seconds
- The expected_lines must be positive integers between 1 and 1000
- File_id strings must be non-empty alphanumeric with hyphens only
- File paths validated for basic format without actual filesystem access
- Python's standard libraries and asyncio are only allowed.
- Solution code must be implemented fully, and should include a driver function for execution
- Solution code should run correctly on an online Python compiler and must be testable

In [ ]:
# code

"""Async log file processor with error simulation and validation."""

import asyncio
from typing import List, Dict, Any

ERROR_TYPE_MAP = {
    "file_not_found": "FileNotFoundError",
    "permission_denied": "PermissionError",
    "timeout": "TimeoutError"
}


def validate_file_id(file_id: str) -> bool:
    """Validates file ID format."""
    return bool(file_id) and all(
        c.isalnum() or c == '-' for c in file_id
    )


def validate_file_path(file_path: str) -> bool:
    """Validates file path format."""
    return (
        isinstance(file_path, str)
        and file_path.startswith('/')
        and '/' in file_path[1:]
    )


def validate_expected_lines(expected_lines: Any) -> bool:
    """Checks if expected lines are within valid range."""
    return isinstance(expected_lines, int) and 1 <= expected_lines <= 1000


def normalize_error_type(error_type: Any) -> str:
    """Returns normalized error type if valid."""
    if error_type in ERROR_TYPE_MAP:
        return error_type
    return None


async def process_log_file(cfg: Dict[str, Any]) -> Dict[str, Any]:
    """Processes a single log file config asynchronously."""
    file_id = cfg.get("file_id")
    file_path = cfg.get("file_path")
    timeout = cfg.get("timeout")
    expected_lines = cfg.get("expected_lines")
    error_type = normalize_error_type(cfg.get("error_type"))

    if (
        not validate_file_id(file_id)
        or not validate_file_path(file_path)
        or not validate_expected_lines(expected_lines)
    ):
        return {
            "file_id": file_id,
            "status": "error",
            "error_type": "FileNotFoundError"
        }

    if not isinstance(timeout, (int, float)) or timeout <= 0.0:
        error_type = "timeout"

    async def simulate_success():
        await asyncio.sleep(min(timeout, 0.05))
        return {
            "file_id": file_id,
            "status": "success",
            "line_count": expected_lines
        }

    async def simulate_error(error_type):
        await asyncio.sleep(min(timeout, 0.01))
        return {
            "file_id": file_id,
            "status": "error",
            "error_type": ERROR_TYPE_MAP[error_type]
        }

    if error_type == "file_not_found":
        return await simulate_error("file_not_found")
    elif error_type == "permission_denied":
        return await simulate_error("permission_denied")
    elif error_type == "timeout":
        return await simulate_error("timeout")
    else:
        try:
            return await asyncio.wait_for(
                simulate_success(), timeout=timeout
            )
        except asyncio.TimeoutError:
            return {
                "file_id": file_id,
                "status": "error",
                "error_type": "TimeoutError"
            }


def analyze_log_files(log_files: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Analyzes multiple log files asynchronously and summarizes results."""
    async def runner():
        log_files_limited = log_files[:50]
        tasks = [process_log_file(cfg) for cfg in log_files_limited]
        results = await asyncio.gather(*tasks)
        total_lines = sum(
            r.get("line_count", 0)
            for r in results if r.get("status") == "success"
        )
        successful_files = sum(
            1 for r in results if r.get("status") == "success"
        )
        error_summary = {
            "FileNotFoundError": sum(
                1 for r in results if r.get("error_type") == "FileNotFoundError"
            ),
            "PermissionError": sum(
                1 for r in results if r.get("error_type") == "PermissionError"
            ),
            "TimeoutError": sum(
                1 for r in results if r.get("error_type") == "TimeoutError"
            ),
        }
        return {
            "total_lines": total_lines,
            "successful_files": successful_files,
            "error_summary": error_summary,
            "file_results": results
        }

    return asyncio.run(runner())


def driver():
    """Driver for running all test scenarios."""
    print("Example 1:")
    log_files1 = [
        {
            "file_id": "srv1",
            "file_path": "/var/log/app.log",
            "timeout": 0.2,
            "expected_lines": 34,
            "error_type": None
        },
        {
            "file_id": "srv2",
            "file_path": "/var/log/err.log",
            "timeout": 0.3,
            "expected_lines": 28,
            "error_type": "file_not_found"
        },
        {
            "file_id": "srv3",
            "file_path": "/var/log/sys.log",
            "timeout": 0.1,
            "expected_lines": 55,
            "error_type": None
        }
    ]
    print(analyze_log_files(log_files1))

    print("\nExample 2:")
    log_files2 = [
        {
            "file_id": "monitor",
            "file_path": "/logs/mon.log",
            "timeout": 0.05,
            "expected_lines": 42,
            "error_type": "timeout"
        },
        {
            "file_id": "backup",
            "file_path": "/logs/bak.log",
            "timeout": 0.4,
            "expected_lines": 18,
            "error_type": "permission_denied"
        }
    ]
    print(analyze_log_files(log_files2))

    print("\nExample 3:")
    log_files3 = [
        {
            "file_id": "web-server-01",
            "file_path": "/logs/web.log",
            "timeout": 0.3,
            "expected_lines": 45,
            "error_type": None
        },
        {
            "file_id": "api-gateway",
            "file_path": "/logs/api.log",
            "timeout": 0.1,
            "expected_lines": 32,
            "error_type": "file_not_found"
        },
        {
            "file_id": "database-01",
            "file_path": "/logs/db.log",
            "timeout": 0.5,
            "expected_lines": 67,
            "error_type": None
        },
        {
            "file_id": "cache-redis",
            "file_path": "/logs/cache.log",
            "timeout": 0.2,
            "expected_lines": 28,
            "error_type": "permission_denied"
        },
        {
            "file_id": "auth-service",
            "file_path": "/logs/auth.log",
            "timeout": 0.4,
            "expected_lines": 44,
            "error_type": None
        }
    ]
    print(analyze_log_files(log_files3))

    print("\nEdge Case: Empty input")
    print(analyze_log_files([]))

    print("\nEdge Case: All succeed")
    log_files4 = [
        {
            "file_id": "a1",
            "file_path": "/a/a.log",
            "timeout": 0.1,
            "expected_lines": 10,
            "error_type": None
        },
        {
            "file_id": "b2",
            "file_path": "/b/b.log",
            "timeout": 0.2,
            "expected_lines": 20,
            "error_type": None
        }
    ]
    print(analyze_log_files(log_files4))

    print("\nEdge Case: All fail")
    log_files5 = [
        {
            "file_id": "x1",
            "file_path": "/x/x.log",
            "timeout": 0.1,
            "expected_lines": 10,
            "error_type": "file_not_found"
        },
        {
            "file_id": "y2",
            "file_path": "/y/y.log",
            "timeout": 0.2,
            "expected_lines": 20,
            "error_type": "permission_denied"
        },
        {
            "file_id": "z3",
            "file_path": "/z/z.log",
            "timeout": 0.0,
            "expected_lines": 30,
            "error_type": None
        }
    ]
    print(analyze_log_files(log_files5))

    print("\nEdge Case: Timeout 0.0 and negative timeout")
    log_files6 = [
        {
            "file_id": "t1",
            "file_path": "/t/t.log",
            "timeout": 0.0,
            "expected_lines": 5,
            "error_type": None
        },
        {
            "file_id": "t2",
            "file_path": "/t/t2.log",
            "timeout": -1.0,
            "expected_lines": 6,
            "error_type": None
        }
    ]
    print(analyze_log_files(log_files6))

    print("\nEdge Case: Duplicate file_id")
    log_files7 = [
        {
            "file_id": "dup",
            "file_path": "/dup/1.log",
            "timeout": 0.1,
            "expected_lines": 7,
            "error_type": None
        },
        {
            "file_id": "dup",
            "file_path": "/dup/2.log",
            "timeout": 0.1,
            "expected_lines": 8,
            "error_type": "file_not_found"
        }
    ]
    print(analyze_log_files(log_files7))

    print("\nEdge Case: Invalid error_type")
    log_files8 = [
        {
            "file_id": "inv",
            "file_path": "/inv/1.log",
            "timeout": 0.1,
            "expected_lines": 9,
            "error_type": "unknown_error"
        }
    ]
    print(analyze_log_files(log_files8))

    print("\nEdge Case: Invalid file_id, file_path, expected_lines")
    log_files9 = [
        {
            "file_id": "",
            "file_path": "/valid.log",
            "timeout": 0.1,
            "expected_lines": 10,
            "error_type": None
        },
        {
            "file_id": "ok",
            "file_path": "invalidpath",
            "timeout": 0.1,
            "expected_lines": 10,
            "error_type": None
        },
        {
            "file_id": "ok2",
            "file_path": "/ok2.log",
            "timeout": 0.1,
            "expected_lines": 0,
            "error_type": None
        }
    ]
    print(analyze_log_files(log_files9))

    print("\nEdge Case: More than 50 files")
    log_files10 = [
        {
            "file_id": f"id{i}",
            "file_path": f"/f{i}.log",
            "timeout": 0.1,
            "expected_lines": 1,
            "error_type": None
        }
        for i in range(60)
    ]
    print(analyze_log_files(log_files10))


if __name__ == "__main__":
    driver()


In [ ]:
# tests

import unittest
import asyncio
import time
from main import analyze_log_files


class TestAnalyzeLogFiles(unittest.TestCase):
    """Test cases for the analyze_log_files function."""

    def setUp(self):
        """Set up test fixtures before each test method."""
        self.maxDiff = None

    def tearDown(self):
        """Clean up after each test method."""

        try:
            loop = asyncio.get_running_loop()
            pending = asyncio.all_tasks(loop)
            for task in pending:
                task.cancel()
        except RuntimeError:

            pass

    def test_example_1_from_problem(self):
        """Test the first example from the problem statement."""
        log_files = [
            {
                "file_id": "srv1",
                "file_path": "/var/log/app.log",
                "timeout": 0.2,
                "expected_lines": 34,
                "error_type": None,
            },
            {
                "file_id": "srv2",
                "file_path": "/var/log/err.log",
                "timeout": 0.3,
                "expected_lines": 28,
                "error_type": "file_not_found",
            },
            {
                "file_id": "srv3",
                "file_path": "/var/log/sys.log",
                "timeout": 0.1,
                "expected_lines": 55,
                "error_type": None,
            },
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 89)
        self.assertEqual(result["successful_files"], 2)
        self.assertEqual(
            result["error_summary"],
            {"FileNotFoundError": 1, "PermissionError": 0, "TimeoutError": 0},
        )
        self.assertEqual(len(result["file_results"]), 3)
        self.assertEqual(
            result["file_results"][0],
            {"file_id": "srv1", "status": "success", "line_count": 34},
        )
        self.assertEqual(
            result["file_results"][1],
            {
                "file_id": "srv2",
                "status": "error",
                "error_type": "FileNotFoundError",
            },
        )
        self.assertEqual(
            result["file_results"][2],
            {"file_id": "srv3", "status": "success", "line_count": 55},
        )

    def test_example_2_from_problem(self):
        """Test the second example from the problem statement."""
        log_files = [
            {
                "file_id": "monitor",
                "file_path": "/logs/mon.log",
                "timeout": 0.05,
                "expected_lines": 42,
                "error_type": "timeout",
            },
            {
                "file_id": "backup",
                "file_path": "/logs/bak.log",
                "timeout": 0.4,
                "expected_lines": 18,
                "error_type": "permission_denied",
            },
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 0)
        self.assertEqual(result["successful_files"], 0)
        self.assertEqual(
            result["error_summary"],
            {"FileNotFoundError": 0, "PermissionError": 1, "TimeoutError": 1},
        )
        self.assertEqual(len(result["file_results"]), 2)
        self.assertEqual(
            result["file_results"][0],
            {
                "file_id": "monitor",
                "status": "error",
                "error_type": "TimeoutError",
            },
        )
        self.assertEqual(
            result["file_results"][1],
            {
                "file_id": "backup",
                "status": "error",
                "error_type": "PermissionError",
            },
        )

    def test_empty_input(self):
        """Test with empty log files list."""
        result = analyze_log_files([])

        self.assertEqual(result["total_lines"], 0)
        self.assertEqual(result["successful_files"], 0)
        self.assertEqual(
            result["error_summary"],
            {"FileNotFoundError": 0, "PermissionError": 0, "TimeoutError": 0},
        )
        self.assertEqual(result["file_results"], [])

    def test_all_files_successful(self):
        """Test when all files are processed successfully."""
        log_files = [
            {
                "file_id": "file1",
                "file_path": "/log1.log",
                "timeout": 0.1,
                "expected_lines": 10,
                "error_type": None,
            },
            {
                "file_id": "file2",
                "file_path": "/log2.log",
                "timeout": 0.1,
                "expected_lines": 20,
                "error_type": None,
            },
            {
                "file_id": "file3",
                "file_path": "/log3.log",
                "timeout": 0.1,
                "expected_lines": 30,
                "error_type": None,
            },
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 60)
        self.assertEqual(result["successful_files"], 3)
        self.assertEqual(result["error_summary"]["FileNotFoundError"], 0)
        self.assertEqual(result["error_summary"]["PermissionError"], 0)
        self.assertEqual(result["error_summary"]["TimeoutError"], 0)

    def test_all_error_types(self):
        """Test with all three error types."""
        log_files = [
            {
                "file_id": "f1",
                "file_path": "/f1.log",
                "timeout": 0.1,
                "expected_lines": 10,
                "error_type": "file_not_found",
            },
            {
                "file_id": "f2",
                "file_path": "/f2.log",
                "timeout": 0.1,
                "expected_lines": 20,
                "error_type": "permission_denied",
            },
            {
                "file_id": "f3",
                "file_path": "/f3.log",
                "timeout": 0.1,
                "expected_lines": 30,
                "error_type": "timeout",
            },
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 0)
        self.assertEqual(result["successful_files"], 0)
        self.assertEqual(result["error_summary"]["FileNotFoundError"], 1)
        self.assertEqual(result["error_summary"]["PermissionError"], 1)
        self.assertEqual(result["error_summary"]["TimeoutError"], 1)

    def test_zero_timeout_causes_timeout(self):
        """Test that zero timeout causes immediate timeout error."""
        log_files = [
            {
                "file_id": "zero_timeout",
                "file_path": "/zero.log",
                "timeout": 0.0,
                "expected_lines": 100,
                "error_type": None,
            }
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 0)
        self.assertEqual(result["successful_files"], 0)
        self.assertEqual(result["error_summary"]["TimeoutError"], 1)
        self.assertEqual(
            result["file_results"][0]["error_type"], "TimeoutError"
        )

    def test_negative_timeout_causes_timeout(self):
        """Test that negative timeout causes immediate timeout error."""
        log_files = [
            {
                "file_id": "neg_timeout",
                "file_path": "/neg.log",
                "timeout": -0.5,
                "expected_lines": 50,
                "error_type": None,
            }
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 0)
        self.assertEqual(result["successful_files"], 0)
        self.assertEqual(result["error_summary"]["TimeoutError"], 1)

    def test_invalid_error_type_treated_as_success(self):
        """Test that invalid error_type is treated as None (success)."""
        log_files = [
            {
                "file_id": "invalid",
                "file_path": "/invalid.log",
                "timeout": 0.1,
                "expected_lines": 75,
                "error_type": "unknown_error",
            }
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 75)
        self.assertEqual(result["successful_files"], 1)
        self.assertEqual(result["file_results"][0]["status"], "success")
        self.assertEqual(result["file_results"][0]["line_count"], 75)

    def test_duplicate_file_ids_processed_independently(self):
        """Test that duplicate file_ids are processed independently."""
        log_files = [
            {
                "file_id": "dup",
                "file_path": "/dup1.log",
                "timeout": 0.1,
                "expected_lines": 10,
                "error_type": None,
            },
            {
                "file_id": "dup",
                "file_path": "/dup2.log",
                "timeout": 0.1,
                "expected_lines": 20,
                "error_type": None,
            },
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 30)
        self.assertEqual(result["successful_files"], 2)
        self.assertEqual(len(result["file_results"]), 2)
        self.assertEqual(result["file_results"][0]["file_id"], "dup")
        self.assertEqual(result["file_results"][1]["file_id"], "dup")

    def test_maintains_original_order(self):
        """Test that results maintain the original input order."""
        log_files = [
            {
                "file_id": "z_last",
                "file_path": "/z.log",
                "timeout": 0.1,
                "expected_lines": 1,
                "error_type": None,
            },
            {
                "file_id": "a_first",
                "file_path": "/a.log",
                "timeout": 0.1,
                "expected_lines": 2,
                "error_type": "file_not_found",
            },
            {
                "file_id": "m_middle",
                "file_path": "/m.log",
                "timeout": 0.1,
                "expected_lines": 3,
                "error_type": None,
            },
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["file_results"][0]["file_id"], "z_last")
        self.assertEqual(result["file_results"][1]["file_id"], "a_first")
        self.assertEqual(result["file_results"][2]["file_id"], "m_middle")

    def test_concurrent_processing(self):
        """Test that files are processed concurrently, not sequentially."""
        log_files = [
            {
                "file_id": "slow1",
                "file_path": "/s1.log",
                "timeout": 0.3,
                "expected_lines": 10,
                "error_type": None,
            },
            {
                "file_id": "slow2",
                "file_path": "/s2.log",
                "timeout": 0.3,
                "expected_lines": 20,
                "error_type": None,
            },
            {
                "file_id": "slow3",
                "file_path": "/s3.log",
                "timeout": 0.3,
                "expected_lines": 30,
                "error_type": None,
            },
        ]

        start_time = time.time()
        result = analyze_log_files(log_files)
        elapsed_time = time.time() - start_time

        self.assertLess(elapsed_time, 0.25)
        self.assertEqual(result["total_lines"], 60)
        self.assertEqual(result["successful_files"], 3)

    def test_mixed_success_and_errors(self):
        """Test with a mix of successful and failed file operations."""
        log_files = [
            {
                "file_id": "success1",
                "file_path": "/s1.log",
                "timeout": 0.1,
                "expected_lines": 100,
                "error_type": None,
            },
            {
                "file_id": "fail1",
                "file_path": "/f1.log",
                "timeout": 0.1,
                "expected_lines": 200,
                "error_type": "file_not_found",
            },
            {
                "file_id": "success2",
                "file_path": "/s2.log",
                "timeout": 0.1,
                "expected_lines": 150,
                "error_type": None,
            },
            {
                "file_id": "fail2",
                "file_path": "/f2.log",
                "timeout": 0.1,
                "expected_lines": 300,
                "error_type": "permission_denied",
            },
            {
                "file_id": "fail3",
                "file_path": "/f3.log",
                "timeout": 0.1,
                "expected_lines": 400,
                "error_type": "timeout",
            },
        ]

        result = analyze_log_files(log_files)

        self.assertEqual(result["total_lines"], 250)
        self.assertEqual(result["successful_files"], 2)
        self.assertEqual(result["error_summary"]["FileNotFoundError"], 1)
        self.assertEqual(result["error_summary"]["PermissionError"], 1)
        self.assertEqual(result["error_summary"]["TimeoutError"], 1)


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/75121/view

#### Model code:

```python
import asyncio
from typing import List, Dict, Any


async def process_file(file_config: Dict[str, Any]) -> Dict[str, Any]:
    """Simulate file processing with potential errors and return a structured result."""
    file_id = file_config["file_id"]
    timeout = file_config["timeout"]
    error_type = file_config.get("error_type", None)

    # Simulate processing and determine potential outcome
    if error_type == "file_not_found":
        return {"file_id": file_id, "status": "error", "error_type": "FileNotFoundError"}
    elif error_type == "permission_denied":
        return {"file_id": file_id, "status": "error", "error_type": "PermissionError"}
    elif isinstance(timeout, (int, float)) and timeout <= 0:
        return {"file_id": file_id, "status": "error", "error_type": "TimeoutError"}
    # <Issue>: Model does not validate file_id, file_path, or expected_lines
    # for constraints (missing validation for format, range, etc.)

    # Simulate successful file read delay with timeout
    try:
        await asyncio.sleep(min(timeout, 2.0))
        return {"file_id": file_id, "status": "success", "line_count": file_config["expected_lines"]}
    except asyncio.TimeoutError:
        return {"file_id": file_id, "status": "error", "error_type": "TimeoutError"}
    # <Issue>: Model does not handle invalid error_type values (should treat
    # unknown error_type as None and process successfully)
    # <Issue>: Model's timeout simulation is not robust; actual timeout errors
    # are not raised for positive timeouts


async def analyze_log_files_async(log_files: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Run concurrent file processing tasks and gather results."""
    tasks = [process_file(file_config) for file_config in log_files]
    return await asyncio.gather(*tasks)
    # <Issue>: Model does not cap input to 50 files as required by constraints


def analyze_log_files(log_files: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Collect statistics from concurrent log file processing and return summary results."""
    # Run asynchronous log file analysis
    loop = asyncio.get_event_loop()
    file_results = loop.run_until_complete(analyze_log_files_async(log_files))
    # <Issue>: Uses get_event_loop/run_until_complete, which is deprecated and
    # can cause runtime issues; should use asyncio.run


    # Process the result
    total_lines = 0
    successful_files = 0
    error_summary = {"FileNotFoundError": 0, "PermissionError": 0, "TimeoutError": 0}

    for result in file_results:
        if result["status"] == "success":
            successful_files += 1
            total_lines += result.get("line_count", 0)
        elif result["status"] == "error":
            error_summary[result["error_type"]] += 1

    return {
        "total_lines": total_lines,
        "successful_files": successful_files,
        "error_summary": error_summary,
        "file_results": file_results
    }


# Example Usage:
if __name__ == "__main__":
    log_files = [
        {"file_id": "web-server-01", "file_path": "/logs/web.log", "timeout": 0.3, "expected_lines": 45, "error_type": None},
        {"file_id": "api-gateway", "file_path": "/logs/api.log", "timeout": 0.1, "expected_lines": 32, "error_type": "file_not_found"},
        {"file_id": "database-01", "file_path": "/logs/db.log", "timeout": 0.5, "expected_lines": 67, "error_type": None},
        {"file_id": "cache-redis", "file_path": "/logs/cache.log", "timeout": 0.2, "expected_lines": 28, "error_type": "permission_denied"},
        {"file_id": "auth-service", "file_path": "/logs/auth.log", "timeout": 0.4, "expected_lines": 44, "error_type": None}
    ]

    result = analyze_log_files(log_files)
    print(result)
```